<a href="https://colab.research.google.com/github/ravi-prakash1907/A-tracking-of-COVID-19/blob/master/Problem_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from text_to_pic import *

---

In [9]:
from PIL import Image

def hideStegoLSB(cover='cover_image.jpg', secret='secret_image.jpg', saveAs='stego_image.png'):
  # Load the cover image and the secret image
  cover_image = Image.open(cover)
  secret_image = Image.open(secret)

  # Get the pixel data for the cover image and the secret image
  cover_data = cover_image.load()
  secret_data = secret_image.load()

  pxSaveCount = 4
  pxHideCount = 8-pxSaveCount

  # Iterate over each pixel in the secret image and hide it in the LSB of the cover image
  for i in range(secret_image.size[0]):
    for j in range(secret_image.size[1]):
      ## cover pic
      cover_r, cover_g, cover_b = cover_data[i, j]
      cover_r_bin = format(cover_r, '08b')
      cover_g_bin = format(cover_g, '08b')
      cover_b_bin = format(cover_b, '08b')
      ## secret pic
      r, g, b = secret_data[i, j]
      r_bin = format(r, '08b')
      g_bin = format(g, '08b')
      b_bin = format(b, '08b')
      ## new pic = cover[MSB-4] + secret[LSB-4]
      new_r_bin = cover_r_bin[:pxHideCount] + r_bin[pxHideCount:]
      new_g_bin = cover_g_bin[:pxHideCount] + g_bin[pxHideCount:]
      new_b_bin = cover_b_bin[:pxHideCount] + b_bin[pxHideCount:]
      new_r = int(new_r_bin, 2)
      new_g = int(new_g_bin, 2)
      new_b = int(new_b_bin, 2)
      cover_data[i, j] = (new_r, new_g, new_b)

  #print("secret_data[310,222] = {} | stego_data[310,222] = {}".format(secret_data[310,222],cover_data[310,222]))
  # Save the stego image
  cover_image.save(saveAs)
  return


In [10]:
def recoverStegoLSB(stego='stego_image.png', saveAs='recovered_secret_image.png'):
  # Load the stego image
  stego_image = Image.open(stego)

  # Create a new image to hold the recovered secret image
  secret_image = Image.new('RGB', stego_image.size)

  # Get the pixel data for the stego image and the secret image
  stego_data = stego_image.load()
  secret_data = secret_image.load()

  pxHideCount = 4
  
  # Iterate over each pixel in the secret image and recover the LSB from the stego image
  for i in range(secret_image.size[0]):
    for j in range(secret_image.size[1]):
      stego_r, stego_g, stego_b = stego_data[i, j]
      stego_r_bin = format(stego_r, '08b')
      stego_g_bin = format(stego_g, '08b')
      stego_b_bin = format(stego_b, '08b')
      new_r_bin = stego_r_bin[pxHideCount:]*2
      new_g_bin = stego_g_bin[pxHideCount:]*2
      new_b_bin = stego_b_bin[pxHideCount:]*2
      new_r = int(new_r_bin, 2)
      new_g = int(new_g_bin, 2)
      new_b = int(new_b_bin, 2)
      secret_data[i, j] = (new_r, new_g, new_b)

  # Save the recovered secret image
  secret_image.save(saveAs)
  return


---

In [8]:
#!mkdir newPics
#!mkdir secrets
#!mkdir recovered

!rm newPics/*
!rm secrets/*
!rm recovered/*

In [18]:
def setSecret(cover,msg='Hi'):
  SEED = 221000
  for i in range(1,62):
    count = str(SEED + i)
    info = msg + ' ' + count + '!'
    ###########################
    picName = count + '.png'
    fName = './newPics/' + picName
    
    # generating image
    getImage(text=info, saveAs=fName)
    # hiding secret
    secretName = './secrets/' + picName
    hideStegoLSB(cover=cover, secret=fName, saveAs=secretName)
  return

In [23]:
# recover original
def recoverPics(getFromDir='./newPics/', saveToDir='./recovered/'):
  SEED = 221000
  for i in range(1,62):
    count = str(SEED + i)
    picName = count + '.png'
    ####
    stego = getFromDir + picName
    recName = saveToDir + picName
    ####
    recoverStegoLSB(stego,recName)
  return

---

In [19]:
# generate all pics
# setSecret('./cover.jpg','Solved by')

In [24]:
recoverPics()